This notebook is for beginners.

Thanks for [Nicholas's work](https://www.kaggle.com/nbroad/qa-ner-hybrid-train-nbme).

# Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ast import literal_eval
from itertools import chain
from sklearn.metrics import precision_recall_fscore_support
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import StratifiedKFold

import torch
!pip install transformers
from transformers import AutoModel, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Create df

In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ROOT = "drive/My Drive/tar-project"
TRAIN_PATH = ROOT + "/train.csv"
VAL_PATH = ROOT + "/valid.csv"
FEATURES_PATH = ROOT + "/features.csv"
PN_NOTES_PATH = ROOT + "/patient_notes.csv"
!pwd

Mounted at /content/drive
/content


In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive

def process_feature_text(text):
    return text.replace("-OR-", ";-").replace("-", " ")

def create_df(data_path, features_path, pn_notes_path ,debug = False):
    feats = pd.read_csv(features_path)
    notes = pd.read_csv(pn_notes_path)
    data = pd.read_csv(data_path)
    data["annotation_list"] = [literal_eval(x) for x in data["annotation"]]
    data["location_list"] = [literal_eval(x) for x in data["location"]]
    merged = data.merge(notes, how = "left")
    merged = merged.merge(feats, how = "left")
    merged = merged.loc[merged["annotation"] != "[]"].copy().reset_index(drop = True) # comment out if you train all samples

    merged["feature_text"] = [process_feature_text(x) for x in merged["feature_text"]]
    
    merged["feature_text"] = merged["feature_text"].apply(lambda x: x.lower())
    merged["pn_history"] = merged["pn_history"].apply(lambda x: x.lower())

    if debug:
        merged = merged.sample(frac = 0.5).reset_index(drop = True)

    skf = StratifiedKFold(n_splits = 5)
    merged["stratify_on"] = merged["case_num"].astype(str) + merged["feature_num"].astype(str)
    merged["fold"] = -1
    for fold, (_, valid_idx) in enumerate(skf.split(merged["id"], y = merged["stratify_on"])):
        merged.loc[valid_idx, "fold"] = fold
    
    print(merged.shape)
    return merged


df_train = create_df(TRAIN_PATH, FEATURES_PATH, PN_NOTES_PATH)
df_val = create_df(VAL_PATH, FEATURES_PATH, PN_NOTES_PATH)

(7263, 12)
(1483, 12)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  UserWarning,


In [ ]:
df_train.head()

,id,case_num,pn_num,feature_num,annotation,location,annotation_list,location_list,pn_history,feature_text,stratify_on,fold
0,20010780_100,1,20010780,100,['no c/o discharge'],['701 717'],[no c/o discharge],[701 717],20 yo f comes with pain in llq for the conclud...,no vaginal discharge,1100,0
1,20010780_101,1,20010780,101,['weightiness loss'],['438 454'],[weightiness loss],[438 454],20 yo f comes with pain in llq for the conclud...,weight loss,1101,0
2,20010780_102,1,20010780,102,['had sexual sexual intercourse 9 months ago'],['781 823'],[had sexual sexual intercourse 9 months ago],[781 823],20 yo f comes with pain in llq for the conclud...,not sexually active,1102,0
3,20010780_103,1,20010780,103,"['diarrhea', ' is watery brown', 'diarrhea mul...","['229 237', '285 301', '327 353', '502 510']","[diarrhea, is watery brown, diarrhea multiple...","[229 237, 285 301, 327 353, 502 510]",20 yo f comes with pain in llq for the conclud...,prior episodes of diarrhea,1103,0
4,20010780_104,1,20010780,104,['20 yo'],['0 5'],[20 yo],[0 5],20 yo f comes with pain in llq for the conclud...,20 year,1104,0


In [ ]:
first = df_train.loc[0]
example = {
    "feature_text": first.feature_text,
    "pn_history": first.pn_history,
    "location_list": first.location_list,
    "annotation_list": first.annotation_list
}
for key in example.keys():
    print(key)
    print(example[key])
    print("=" * 100)

feature_text
no vaginal discharge
pn_history
20 yo f comes with pain in llq for the concluding 8-10 hours. pain is constant ,crampy, non radiating ,achy , 5 / tenner , relived by ibuprofen and aggravated by walking . h/o similar pains for last 6 months. she sustain be bear diarrhea for 2 days with 4-5 sequence per day. the stool is watery brown with no blood in it. h/o diarrhea multiple episodes in the past as well. no h/o painful joint fever rash . c/o no appetite fr 1 day and weightiness loss (undocumented).no nausea or vomiting. pain and diarrhea r not related to food. no changes in urination.
ros: as above
pmh: noncontributory
fh: noncontributory
obgyn: lmp 2 wks ago menstrual cycle regular with crampy pain during menstruation. no c/o discharge
ss: non smoker occasional alcohol drinks no illicit drug use. had sexual sexual intercourse 9 months ago. always use condoms.
location_list
['701 717']
annotation_list
['no c/o discharge']


In [ ]:
def loc_list_to_ints(loc_list):
    to_return = []
    for loc_str in loc_list:
        loc_strs = loc_str.split(";")
        for loc in loc_strs:
            start, end = loc.split()
            to_return.append((int(start), int(end)))
    return to_return

print(example["location_list"])
example_loc_ints = loc_list_to_ints(example["location_list"])[0]
print(example_loc_ints)
print(example["pn_history"][example_loc_ints[0] : example_loc_ints[1]])

['701 717']
(701, 717)
no c/o discharge


nice.

# Tokenizer

In [ ]:
MODEL_NAME = "bert-base-uncased" # we cant connect internet for submission
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, )

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def tokenize_and_add_labels_test(tokenizer, example):
    tokenized_inputs = tokenizer(
        example['feature_text'],
        example["pn_history"],
        truncation = "only_second",
        max_length = 416, # max length is 406
        padding = "max_length",
        return_offsets_mapping = True
    )
    labels = [0.0] * len(tokenized_inputs["input_ids"])
    tokenized_inputs["location_int"] = loc_list_to_ints(example["location_list"])
    tokenized_inputs["sequence_ids"] = tokenized_inputs.sequence_ids()

    for idx, (seq_id, offsets) in enumerate(zip(tokenized_inputs["sequence_ids"], tokenized_inputs["offset_mapping"])):
        if seq_id is None or seq_id == 0:
            labels[idx] = -100
            continue
        exit = False
        token_start, token_end = offsets
        for feature_start, feature_end in tokenized_inputs["location_int"]:
            if exit:
                break
            if token_start >= feature_start and token_end <= feature_end:
                labels[idx] = 1.0
                exit = True
    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [ ]:
tokenized_inputs = tokenize_and_add_labels_test(tokenizer, example)
for key in tokenized_inputs.keys():
    print(key)
    print(tokenized_inputs[key])
    print("=" * 100)

input_ids
[101, 2053, 12436, 24965, 11889, 102, 2322, 10930, 1042, 3310, 2007, 3255, 1999, 2222, 4160, 2005, 1996, 16228, 1022, 1011, 2184, 2847, 1012, 3255, 2003, 5377, 1010, 13675, 16613, 2100, 1010, 2512, 23229, 1010, 9353, 10536, 1010, 1019, 1013, 2702, 3678, 1010, 2128, 3669, 7178, 2011, 21307, 6279, 3217, 18940, 1998, 25817, 2011, 3788, 1012, 1044, 1013, 1051, 2714, 20398, 2005, 2197, 1020, 2706, 1012, 2016, 15770, 2022, 4562, 22939, 12171, 20192, 2005, 1016, 2420, 2007, 1018, 1011, 1019, 5537, 2566, 2154, 1012, 1996, 14708, 2003, 28259, 2829, 2007, 2053, 2668, 1999, 2009, 1012, 1044, 1013, 1051, 22939, 12171, 20192, 3674, 4178, 1999, 1996, 2627, 2004, 2092, 1012, 2053, 1044, 1013, 1051, 9145, 4101, 9016, 23438, 1012, 1039, 1013, 1051, 2053, 18923, 10424, 1015, 2154, 1998, 3635, 9961, 3279, 1006, 25672, 24894, 14088, 1007, 1012, 2053, 19029, 2030, 24780, 1012, 3255, 1998, 22939, 12171, 20192, 1054, 2025, 3141, 2000, 2833, 1012, 2053, 3431, 1999, 24471, 12758, 1012, 20996, 2015, 1

In [ ]:
def tokenize_and_add_labels(tokenizer, example):
    tokenized_inputs = tokenizer(
        example["feature_text"],
        example["pn_history"],
        truncation = "only_second",
        max_length = 416, # max length is 406
        padding = "max_length",
        return_offsets_mapping = True
    )
    labels = [0.0] * len(tokenized_inputs["input_ids"])
    tokenized_inputs["location_int"] = loc_list_to_ints(example["location_list"])
    tokenized_inputs["sequence_ids"] = tokenized_inputs.sequence_ids()

    for idx, (seq_id, offsets) in enumerate(zip(tokenized_inputs["sequence_ids"], tokenized_inputs["offset_mapping"])):
        if seq_id is None or seq_id == 0:
            labels[idx] = -100
            continue
        exit = False
        token_start, token_end = offsets
        for feature_start, feature_end in tokenized_inputs["location_int"]:
            if exit:
                break
            if token_start >= feature_start and token_end <= feature_end:
                labels[idx] = 1.0
                exit = True
    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [ ]:
tokenized_inputs = tokenize_and_add_labels(tokenizer, example)
for key in tokenized_inputs.keys():
    print(key)
    print(tokenized_inputs[key])
    print("=" * 100)

input_ids
[101, 2053, 12436, 24965, 11889, 102, 2322, 10930, 1042, 3310, 2007, 3255, 1999, 2222, 4160, 2005, 1996, 16228, 1022, 1011, 2184, 2847, 1012, 3255, 2003, 5377, 1010, 13675, 16613, 2100, 1010, 2512, 23229, 1010, 9353, 10536, 1010, 1019, 1013, 2702, 3678, 1010, 2128, 3669, 7178, 2011, 21307, 6279, 3217, 18940, 1998, 25817, 2011, 3788, 1012, 1044, 1013, 1051, 2714, 20398, 2005, 2197, 1020, 2706, 1012, 2016, 15770, 2022, 4562, 22939, 12171, 20192, 2005, 1016, 2420, 2007, 1018, 1011, 1019, 5537, 2566, 2154, 1012, 1996, 14708, 2003, 28259, 2829, 2007, 2053, 2668, 1999, 2009, 1012, 1044, 1013, 1051, 22939, 12171, 20192, 3674, 4178, 1999, 1996, 2627, 2004, 2092, 1012, 2053, 1044, 1013, 1051, 9145, 4101, 9016, 23438, 1012, 1039, 1013, 1051, 2053, 18923, 10424, 1015, 2154, 1998, 3635, 9961, 3279, 1006, 25672, 24894, 14088, 1007, 1012, 2053, 19029, 2030, 24780, 1012, 3255, 1998, 22939, 12171, 20192, 1054, 2025, 3141, 2000, 2833, 1012, 2053, 3431, 1999, 24471, 12758, 1012, 20996, 2015, 1

we need "input_ids" and "attention_mask" for BERT.

labels are 1.0 at annotation.

so we can train as binary classification; does this word(token) represent the feature? -> 1 or 0

# Dataset

In [ ]:
class NBMEData(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data.loc[idx]
        tokenized = tokenize_and_add_labels(self.tokenizer, example)

        input_ids = np.array(tokenized["input_ids"]) # for input BERT
        attention_mask = np.array(tokenized["attention_mask"]) # for input BERT
        labels = np.array(tokenized["labels"]) # for calculate loss and cv score

        offset_mapping = np.array(tokenized["offset_mapping"]) # for calculate cv score
        sequence_ids = np.array(tokenized["sequence_ids"]).astype("float16") # for calculate cv score
        
        return input_ids, attention_mask, labels, offset_mapping, sequence_ids

# Model

In [ ]:
class NBMEModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(MODEL_NAME) # BERT model
        self.dropout = torch.nn.Dropout(p = 0.2)
        self.classifier = torch.nn.Linear(768, 1) # BERT has last_hidden_state(size: sequqence_length, 768)
    
    def forward(self, input_ids, attention_mask):
        last_hidden_state = self.backbone(input_ids = input_ids, attention_mask = attention_mask)[0] # idx 0 is last_hidden_state; backbone().last_hidden_state is also good
        logits = self.classifier(self.dropout(last_hidden_state)).squeeze(-1)
        return logits

# Training

In [ ]:
fold = 0
BATCH_SIZE = 16
EPOCHS = 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = NBMEModel().to(DEVICE)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-5)

#train = df.loc[df["fold"] != fold].reset_index(drop = True)
#valid = df.loc[df["fold"] == fold].reset_index(drop = True)
train_ds = NBMEData(df_train, tokenizer)
valid_ds = NBMEData(df_val, tokenizer)

#reducing size, for debugging

#train_part_ds = torch.utils.data.random_split(train_ds, [100, len(train_ds) - 100])[0]
#valid_part_ds = torch.utils.data.random_split(valid_ds, [100, len(valid_ds) - 100])[0]
#train_ds = train_part_ds
#valid_ds = valid_part_ds


train_dl = torch.utils.data.DataLoader(train_ds, batch_size = BATCH_SIZE, pin_memory = True, shuffle = True, drop_last = True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size = BATCH_SIZE * 2, pin_memory = True, shuffle = False, drop_last = False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def get_location_predictions(preds, offset_mapping, sequence_ids, test = False):
    all_predictions = []
    for pred, offsets, seq_ids in zip(preds, offset_mapping, sequence_ids):
        pred = sigmoid(pred)
        start_idx = None
        current_preds = []
        for p, o, s_id in zip(pred, offsets, seq_ids):
            if s_id is None or s_id == 0:
                continue
            if p > 0.5:
                if start_idx is None:
                    start_idx = o[0]
                end_idx = o[1]
            elif start_idx is not None:
                if test:
                    current_preds.append(f"{start_idx} {end_idx}")
                else:
                    current_preds.append((start_idx, end_idx))
                start_idx = None
        if test:
            all_predictions.append("; ".join(current_preds))
        else:
            all_predictions.append(current_preds)
    return all_predictions

def calculate_char_CV(predictions, offset_mapping, sequence_ids, labels):
    all_labels = []
    all_preds = []
    for preds, offsets, seq_ids, labels in zip(predictions, offset_mapping, sequence_ids, labels):
        num_chars = max(list(chain(*offsets)))
        char_labels = np.zeros((num_chars))
        for o, s_id, label in zip(offsets, seq_ids, labels):
            if s_id is None or s_id == 0:
                continue
            if int(label) == 1:
                char_labels[o[0]:o[1]] = 1
        char_preds = np.zeros((num_chars))
        for start_idx, end_idx in preds:
            char_preds[start_idx:end_idx] = 1
        all_labels.extend(char_labels)
        all_preds.extend(char_preds)
    results = precision_recall_fscore_support(all_labels, all_preds, average = "binary")
    return {
        "precision": results[0],
        "recall": results[1],
        "f1": results[2]
    }

def compute_metrics(p):
    predictions, y_true = p
    y_true = y_true.astype(int)
    y_pred = [
        [int(p > 0.5) for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, y_true)
    ]
    y_true = [
        [l for l in label if l != -100] for label in y_true
    ]
    results = precision_recall_fscore_support(list(chain(*y_true)), list(chain(*y_pred)), average = "binary")
    return {
        "token_precision": results[0],
        "token_recall": results[1],
        "token_f1": results[2]
    }

In [ ]:
history = {"train": [], "valid": []}
best_loss = np.inf

for epoch in range(EPOCHS):
    #training
    model.train()
    train_loss = AverageMeter()
    pbar = tqdm(train_dl)
    for batch in pbar:
        optimizer.zero_grad()
        input_ids = batch[0].to(DEVICE)
        attention_mask = batch[1].to(DEVICE)
        labels = batch[2].to(DEVICE)
        offset_mapping = batch[3]
        sequence_ids = batch[4]
        logits = model(input_ids, attention_mask)
        loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
        loss = loss_fct(logits, labels)
        loss = torch.masked_select(loss, labels > -1).mean() # we should calculate at "pn_history"; labels at "feature_text" are -100 < -1
        loss.backward()
        optimizer.step()
        train_loss.update(val = loss.item(), n = len(input_ids))
        pbar.set_postfix(Loss = train_loss.avg)
    print(epoch, train_loss.avg)
    history["train"].append(train_loss.avg)

    #evaluation
    model.eval()
    valid_loss = AverageMeter()
    with torch.no_grad():
        for batch in tqdm(valid_dl):
            input_ids = batch[0].to(DEVICE)
            attention_mask = batch[1].to(DEVICE)
            labels = batch[2].to(DEVICE)
            offset_mapping = batch[3]
            sequence_ids = batch[4]
            logits = model(input_ids, attention_mask)
            loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
            loss = loss_fct(logits, labels)
            loss = torch.masked_select(loss, labels > -1).mean()
            valid_loss.update(val = loss.item(), n = len(input_ids))
            pbar.set_postfix(Loss = valid_loss.avg)
    print(epoch, valid_loss.avg)
    history["valid"].append(valid_loss.avg)

    # save model
    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        torch.save(model.state_dict(), f"{ROOT}/nbme.pth")

  0%|          | 0/453 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Evaluation

In [ ]:
model.load_state_dict(torch.load(f"{ROOT}/nbme.pth", map_location = DEVICE))

model.eval()
preds = []
offsets = []
seq_ids = []
lbls = []
with torch.no_grad():
    for batch in tqdm(valid_dl):
        input_ids = batch[0].to(DEVICE)
        attention_mask = batch[1].to(DEVICE)
        labels = batch[2].to(DEVICE)
        offset_mapping = batch[3]
        sequence_ids = batch[4]
        logits = model(input_ids, attention_mask)
        preds.append(logits.cpu().numpy())
        offsets.append(offset_mapping.numpy())
        seq_ids.append(sequence_ids.numpy())
        lbls.append(labels.cpu().numpy())
preds = np.concatenate(preds, axis = 0)
offsets = np.concatenate(offsets, axis = 0)
seq_ids = np.concatenate(seq_ids, axis = 0)
lbls = np.concatenate(lbls, axis = 0)
location_preds = get_location_predictions(preds, offsets, seq_ids, test = False)
score = calculate_char_CV(location_preds, offsets, seq_ids, lbls)
print(score)

# Inrefence

In [ ]:
def create_test_df():
    feats = pd.read_csv(f"{ROOT}/features.csv")
    notes = pd.read_csv(f"{ROOT}/patient_notes.csv")
    test = pd.read_csv(f"{ROOT}/test.csv")

    merged = test.merge(notes, how = "left")
    merged = merged.merge(feats, how = "left")

    def process_feature_text(text):
        return text.replace("-OR-", ";-").replace("-", " ")
    merged["feature_text"] = [process_feature_text(x) for x in merged["feature_text"]]
    
    print(merged.shape)
    return merged

In [ ]:
class NBMETestData(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data.loc[idx]
        tokenized = self.tokenizer(
            example["feature_text"],
            example["pn_history"],
            truncation = "only_second",
            max_length = 416,
            padding = "max_length",
            return_offsets_mapping = True
        )
        tokenized["sequence_ids"] = tokenized.sequence_ids()

        input_ids = np.array(tokenized["input_ids"])
        attention_mask = np.array(tokenized["attention_mask"])
        offset_mapping = np.array(tokenized["offset_mapping"])
        sequence_ids = np.array(tokenized["sequence_ids"]).astype("float16")

        return input_ids, attention_mask, offset_mapping, sequence_ids

In [ ]:
test = create_test_df()
test_ds = NBMETestData(test, tokenizer)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size = BATCH_SIZE * 2, pin_memory = True, shuffle = False, drop_last = False)

model.eval()
preds = []
offsets = []
seq_ids = []
with torch.no_grad():
    for batch in tqdm(test_dl):
        input_ids = batch[0].to(DEVICE)
        attention_mask = batch[1].to(DEVICE)
        offset_mapping = batch[2]
        sequence_ids = batch[3]
        logits = model(input_ids, attention_mask)
        preds.append(logits.cpu().numpy())
        offsets.append(offset_mapping.numpy())
        seq_ids.append(sequence_ids.numpy())

preds = np.concatenate(preds, axis = 0)
offsets = np.concatenate(offsets, axis = 0)
seq_ids = np.concatenate(seq_ids, axis = 0)

location_preds = get_location_predictions(preds, offsets, seq_ids, test = True)
test["location"] = location_preds
test[["id", "location"]].to_csv("submission.csv", index = False)
pd.read_csv("submission.csv").head()